## CV11 - OCR 기술의 개요
OCR = Text detection + Text recognition
### Before Deep Learning
OCR의 역사: https://en.wikipedia.org/wiki/Timeline_of_optical_character_recognition  
딥러닝 이전에 이미지 속 문자를 찾아내는 방식: https://arxiv.org/pdf/1811.06193.pdf  
위 논문에서 OCR엔진으로 Tesseract OCR를 사용하였고 1985년부터 개발되어 2006년부턴 구글의 후원을 받고 있다.  
최근엔 LSTM을 적용하는 등 딥러닝 베이스 아키텍처로 발전하고 있다.  
Tesseract OCR을 통한 4단계 과정  
1. Adaptive Thresholding: 입력 영상의 이진화를 통해 흑백처럼 변환
2. Connected Component Analysis: 문자 영역 검출
3. Find Lines and Words: 라인 또는 워드 단위 추출
4. Recognize Word: 문자를 하나씩 인식하고 다시 결합하는 과정

### Text detection
이미지에서 텍스트를 검출할 때는 검출하기 위한 최소 단위를 정해야 한다. (e.g. 문장 단위 혹은 단어, 글자 단위)  
문장, 단어일 경우 길이가 길거나 짧은 문장이나 단어를 찾아내야 한다.  
글자 단위일 경우 모든 텍스트를 찾아낼 수 있지만 이를 다시 단어나 문장으로 맥락에 맞게 이어줘야 한다.  
다양한 Text detection: https://arxiv.org/pdf/1704.03155v2.pdf  
단어 단위의 탐지는 anchor를 정의하고 단어의 유무, bbox의 크기를 추정하여 단어를 찾아낸다. object detection  
글자 단위의 탐지는 글자 영역을 segmentation하는 방법으로 접근한다.  
1. Regression
TextBoxes: https://arxiv.org/pdf/1611.06779.pdf  
이전에는 글자 단위로 인식한 후 결합하는 형태였지만 이 논문에서는 딥러닝을 이용하여 단어 단위로 인식한다. SSD  
보통의 bbox와 달리 텍스트의 특징에 맞게 가로로 긴 bbox를 설정한다. 그리고 촙촙하게 detection하기 위해  
vertical offset를 적용하여 셀과 셀사이에 bbox를 설정한다.  
2. Segmentation
PixelLink: https://arxiv.org/pdf/1801.01315.pdf  
semantic segmentation과 같이 이미지 내에서 객체인 문자과 배경을 분리하고 text가 어느 방향으로 연결되었는지  
같이 학습을 하여 text간 분리 및 연결을 할수 있는 정보를 추가적으로 활용한다.
output중 하나는 text/non-text를 예측하는 2개의 커널을 가진다.  
나머지 8개는 인접한 8개의 pixel에 대한 연결여부를 의미하는 16개의 커널로 이루어진 link prediction map이다.  
conv 1x1, 2(16)레이어는 인접 pixel간 연결 구조가 지속적으로 유지되도록 한다. unet처럼 concat한다.  
3. 최근의 방법들
   CRAFT: https://arxiv.org/abs/1904.01941
   character단위로 문자의 위치를 찾아낸 뒤 이를 연결하는 방식, segmentation기반으로 구현.  
   문자영역을 가우시안 분포를 따르는 원형의 score map를 만들어서 배치시키는 방법으로 학습한다.  
   문자 단위의 라벨을 가진 데이터셋이 충분하지 않아서 단어 영역에 inference한 후 얻어진 문자 단위의 위치를  
   다시 학습에 활용한 weakly supervised learning
   
   Pyramid Mask Text Detector: Mask-RCNN 활용, text영역을 region proposal network로 찾아내서  
   box head를 정확하게 regression, classification하고 Mask head에서 instance segmentation한다.  
   단어의 사각형 배치 특성을 반영하여 피라미드 형태의 score map을 활용  

### Text recognition
CNN + RNN = CRNN?  
https://arxiv.org/pdf/1507.05717.pdf  
unsegmented data의 특징 중에 하나는 segment되어 있지 않은 하위 데이터들끼리 시퀀스를 이루고 있다는 점.  
문자이미지에서 정보를 추출하기 위해 feature extractor가 필요하다. 이는 CNN으로 가능하다.  
feature을 map to sequence을 통해 sequence형태의 feature로 변환한 후 RNN에 입력한다.  
output은 24개의 글자로 이루어짐.  이를 실제 글자로 바꾸기 위해 CTC를 활용한다.  

CTC: http://www.cs.toronto.edu/~graves/icml_2006.pdf  
만약 'HHHHEEEELLLOOOOOO...'를 'HELLO'로 만들기 위해서는 중복된 글자를 한 글자로 바꿔야 한다.  
하지만 'HELO'가 되기 때문에 'L'이 중복된 경우 구별해야 하는데 blank를 넣어줘서 구분한다. HEL-LO  
blank('-')토큰을 넣어준 뒤 decoder에서 지워주면 L-L이 LL로 바뀐다.  

TPS: https://arxiv.org/pdf/1603.03915.pdf  
휘어지거나 불규칙한 방향의 글자를 정방향으로 맞춰준다.  
spatial transformer network를 이용하여 미리 정의된 20개의 control point가 얼마나 움직여야 하는지 네트워크를 통해 예측한다.  
spatial transformer network: https://3months.tistory.com/197
### Text recognition + Attention
Attention, transformer 
1. Attention sequence prediction
   Robust Scene Text Recognition with Automatic Rectification  
   https://arxiv.org/pdf/1603.03915.pdf  
   Focusing Attention: https://arxiv.org/pdf/1709.02054.pdf  
2. Transformer
   A Simple and Strong Convolutional-Attention Network for Irregular text Recognition  
   https://arxiv.org/pdf/1904.01375.pdf  
   https://hulk89.github.io/machine%20learning/2019/05/15/A-Simple-and-Robust-Convolutional-Attention-Network-For-Irregular-Text-Recognition/  
   
Attention의 핵심은 Decoder에서 현재 스텝에서 중요한 Encoder의 state에 가중치를 높게 준다는 점이다.  